In [ ]:
import os, sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import openpyxl
import xlrd
from sklearn.preprocessing import normalize, StandardScaler, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from nilearn.connectome import ConnectivityMeasure
from pandas import DataFrame
import scipy as sc
from scipy import io
from scipy.stats import pearsonr
from os.path import join, exists, dirname, basename
from glob import glob
from brainspace import gradient
from random import randint
import nibabel as nib
import seaborn as sns

# Load HCP file, BNA atlas mapping

In [ ]:
import time
import nibabel as nib


Atlas = nib.load(join(atlas_path,'BNA_2mm.nii')).get_fdata()
n_roi = int(np.max(Atlas))
print(n_roi)
print(Atlas.shape)

save_ts = False

for p in path_subj:
    print(f'{path_subj.index(p)} / {len(path_subj)}')

    subj_id = basename(p)
    path_REST = glob(join(p,'MNINonLinear/Results/rfMRI_REST*'))
    
    for f in path_REST:
        start_time = time.time()
        save_path = f
        
        print(f' {subj_id} {basename(f)}')
        
        nii_file = glob(join(f,'*hp2000_clean.nii.gz'))
        fmri = nib.load(nii_file[0]).get_fdata()

        ts = np.array([np.mean(fmri[Atlas==roi], axis=0) for roi in range(1, n_roi+1)]) 
        if save_ts:
            print(ts.shape)
            np.save(join(save_path, f'ts_BNA.npy'), ts)
            print(' save ts')
        
        print(f' {time.time() - start_time} seconds!\n')

# Load HCP BNA time series, demean

In [ ]:
import time
import nibabel as nib

for p in path_subj:
    print(f'{path_subj.index(p)} / {len(path_subj)}')

    subj_id = basename(p)
    path_REST = glob(join(p,'MNINonLinear/Results/rfMRI_REST*'))
    
    for f in path_REST:
        start_time = time.time()
        save_path = f
        
        print(f' {subj_id} {basename(f)}')
        
        ts_file = glob(join(f, 'ts_BNA.npy'))
        ts = np.load(ts_file[0])
        ts_demean = np.subtract(ts, ts.mean(axis=1).reshape(-1,1))
        np.save(join(save_path,'ts_demean_BNA.npy'),ts_demean)
        print(' save ts demean')
        print(f' {time.time() - start_time} seconds!\n')

# Concat HCP not twin times series, make FC

In [ ]:
template_demo = pd.read_excel(join(main_path, 'S1200_total.xlsx'), sheet_name='NotTwin_QC', skiprows=0)

sub_list_hcp = template_demo['Subject']

In [ ]:
sub_list_hcp

In [ ]:
# check REST quality

for i in sub_list_hcp:
    REST_list = glob(join(path_subj, f'{i}/MNINonLinear/Results/*'))
    
    if len(REST_list) !=4:
        print(i, len(REST_list))
print('QC finish')

In [ ]:
import time

for i in sub_list_hcp:
    print(f'{np.where(sub_list == i)[0][0]} / {len(sub_list_hcp)}, {i}')
    
    start_time = time.time()
    
    folder_name = str(i)
    if not(os.path.isdir(join(save_path,f'{folder_name}'))):
        os.makedirs(os.path.join(save_path,f'{folder_name}'))
    
    REST_list = sorted(glob(join(path_subj, f'{i}/MNINonLinear/Results/*')))
    LR1 = np.load(f'{REST_list[0]}/ts_demean_BNA.npy')
    RL1 = np.load(f'{REST_list[1]}/ts_demean_BNA.npy')
    LR2 = np.load(f'{REST_list[2]}/ts_demean_BNA.npy')
    RL2 = np.load(f'{REST_list[3]}/ts_demean_BNA.npy')
    
    ts_demean_stacked = np.concatenate((LR1,RL1,LR2,RL2), axis=1)
    conn_mat =np.nan_to_num(np.where(np.eye(n_roi) ==1, 0, np.corrcoef(ts_demean_stacked)))
    conn_mat_rtoz =  np.nan_to_num(np.arctanh(conn_mat),nan=0.0)
    
    np.save(join(save_path,folder_name,'ts_demean_stacked_BNA.npy'), ts_demean_stacked)
    np.save(join(save_path,folder_name,'conn_mat_pear_BNA.npy'), conn_mat)
    np.save(join(save_path,folder_name,'conn_mat_pear_rtoz_BNA.npy'), conn_mat_rtoz)
    
    print(' ts stacked')
    print(' conn mat')
    print(' conn mat r to z')
    
    print(f'{time.time() - start_time} seconds!\n')

# Make gradient (eigenvector) template (using HCP)

In [ ]:
# Load only surface cortex
file_list_hcp = [np.load(join(save_path, str(i), 'conn_mat_pear_BNA.npy'))[:210,:210] for i in sub_list_hcp]
    
grp_conn_mat = np.array(file_list_hcp).mean(axis=0)
grp_conn_mat_rtoz = np.nan_to_num(np.arctanh(grp_conn_mat))

In [ ]:
# FC Figure
plt.figure(figsize = (5,5)) # (20,20)
plt.matshow(grp_conn_mat,  fignum=1)

In [ ]:
for sparsity in [0.5, 0.8, 0.75, 0.7, 0.6]:

#     sparsity = 0.9

    k = str(int(100-sparsity*100))
    k = k.zfill(2)

    print(f'Top {k}')

    noaff_grp_conn_mat = gradient.compute_affinity(grp_conn_mat_rtoz, sparsity = sparsity)
    aff_grp_conn_mat = gradient.compute_affinity(grp_conn_mat_rtoz, kernel = 'cosine', sparsity = sparsity)

    print(f'noaff isnan : {np.isnan(noaff_grp_conn_mat).sum()}, symmetric : {gradient.is_symmetric(noaff_grp_conn_mat)}')
    print(f'aff isnan : {np.isnan(aff_grp_conn_mat).sum()}, symmetric : {gradient.is_symmetric(aff_grp_conn_mat)}')

    # rmsubcor = remove subcortical cortex

    np.save(join(save_path, f'noaff_grp_conn_mat_pear_top{k}_BNA_rmsubcor.npy'), noaff_grp_conn_mat)
    np.save(join(save_path, f'aff_grp_conn_mat_pear_top{k}_BNA_rmsubcor.npy'), aff_grp_conn_mat)

In [ ]:
comp_num = 10

# grp_aff = np.load(join(save_path, f'noaff_grp_conn_mat_pear_top10_BNA_rmsubcor.npy'))
grp_aff = np.load(join(save_path, f'aff_grp_conn_mat_pear_top10_BNA_rmsubcor.npy'))

# emb = gradient.embedding.PCAMaps(n_components = comp_num)
emb = gradient.embedding.DiffusionMaps(n_components = comp_num)

emb.fit(grp_aff)
ref_lam = emb.lambdas_ 
ref_PC = emb.maps_

grad1 = ref_PC[:,0]
grad2 = ref_PC[:,1]
grad3 = ref_PC[:,2]

# Load Validation datset (Lemon) time series, demean

In [ ]:
import time

Atlas = nib.load(join(atlas_path,'BNA_2mm.nii')).get_fdata()
n_roi = int(np.max(Atlas))
print(n_roi)
print(Atlas.shape)

save_ts = True

for p in path_subj[156:]:
    print(f'{path_subj.index(p)} / {len(path_subj)}')
    
    start_time = time.time()

    subj_id = basename(p)
    folder_name = subj_id
    print(subj_id)
    
    nii_file = glob(join(p, 'func','*MNI2mm*'))
    
    if not nii_file:
        print('Pass')
        continue
    
    if not (os.path.isdir(os.path.join(save_path_lemon,f'{folder_name}'))):
            os.makedirs(os.path.join(save_path_lemon,f'{folder_name}'))
    
    
    fmri = nib.load(nii_file[0]).get_fdata()
    
    ts = np.array([np.mean(fmri[Atlas==roi], axis=0) for roi in range(1, n_roi+1)]) 
    ts_demean = np.subtract(ts, ts.mean(axis=1).reshape(-1,1))
    
    conn_mat =np.nan_to_num(np.where(np.eye(n_roi) ==1, 0, np.corrcoef(ts_demean)))
    conn_mat_rtoz =  np.nan_to_num(np.arctanh(conn_mat),nan=0.0)
        
    if save_ts:
        print(ts.shape)
        np.save(join(save_path_lemon,folder_name, f'ts_BNA.npy'), ts)
        print(' save ts')
        np.save(join(save_path_lemon,folder_name,'ts_demean_BNA.npy'),ts_demean)
        print(' save ts demean')
        np.save(join(save_path_lemon,folder_name,'conn_mat_pear_BNA.npy'), conn_mat)
        print(' conn mat')
        np.save(join(save_path_lemon,folder_name,'conn_mat_pear_rtoz_BNA.npy'), conn_mat_rtoz)
        print(' conn mat r to z')
        
    print(f' {time.time() - start_time} seconds!\n')


# Make gradient Lemon

In [ ]:
demo_lemon = pd.read_excel(join(main_path, 'LEMON_Demographic.xlsx'), sheet_name='n=212', skiprows=1, usecols='A:Z')
sub_list_lemon = demo_lemon['ID']

file_list = np.array([glob(join(main_path,'data_Lemon',i)) for i in sub_list_lemon]).flatten()

len(file_list)

In [ ]:
for sparsity in [0.9]:

#     sparsity = 0.75 # 0.75 0.5 0.8 0.7 0.6

    k = str(int(100-sparsity*100))
    k = k.zfill(2)

    print(f'Top {k}')

    folder_name = f'top{k}'

    for i in file_list:
        print(f'{np.where(file_list==i)[0].item()} / {len(file_list)}')

        if not (os.path.isdir(os.path.join(i,folder_name))):
            os.makedirs(os.path.join(i,folder_name))
        else:
            pass

        conn_mat = np.load(join(i,'conn_mat_pear_rtoz_BNA.npy'))[:210, :210]

        noaff_conn_mat = gradient.compute_affinity(conn_mat, sparsity = sparsity)
        aff_conn_mat = gradient.compute_affinity(conn_mat, kernel = 'cosine', sparsity = sparsity)

        print(f'noaff isnan : {np.isnan(noaff_conn_mat).sum()}, symmetric : {gradient.is_symmetric(noaff_conn_mat)}')
        print(f'aff isnan : {np.isnan(aff_conn_mat).sum()}, symmetric : {gradient.is_symmetric(aff_conn_mat)}')

        np.save(join(i, folder_name, f'noaff_conn_mat_pear_top{k}_BNA_rmsubcor.npy'), noaff_conn_mat)
        np.save(join(i, folder_name, f'aff_conn_mat_pear_top{k}_BNA_rmsubcor.npy'), aff_conn_mat)

In [ ]:
for i in [10]:

    Topk = str(i) # 25 50 20 30 40
    folder = f'top{Topk}'

    grp_noaff = np.load(join(save_path, f'noaff_grp_conn_mat_pear_top{Topk}_BNA_rmsubcor.npy'))
    grp_aff = np.load(join(save_path, f'aff_grp_conn_mat_pear_top{Topk}_BNA_rmsubcor.npy'))

    comp_num = 10

    # PCA
    print('PCA')
    emb_pca = gradient.embedding.PCAMaps(n_components = comp_num)
    emb_pca.fit(grp_noaff)
    ref_lam_pca = emb_pca.lambdas_ 
    ref_PC_pca = emb_pca.maps_ 

    for i in file_list:
        print(f'{np.where(file_list==i)[0].item()}', '', end='', flush=True)
        dat = np.load(join(i,folder,f'noaff_conn_mat_pear_top{Topk}_BNA_rmsubcor.npy'))

        emb_pca.fit(dat)
        lam, grad = [None]*1, [None]*1
        lam[0], grad[0] = emb_pca.lambdas_ , emb_pca.maps_

        pa = gradient.ProcrustesAlignment(n_iter=10)
        pa.fit(grad, reference=ref_PC_pca)
        aligned = np.array(pa.aligned_)

        np.save(join(i,folder,f'gradients_PCA_top{Topk}_BNA_rmsubcor.npy'), aligned)

    print('\n')

    # Diffusion map
    print('Diffusion Map')
    emb_dm = gradient.embedding.DiffusionMaps(n_components = comp_num)
    emb_dm.fit(grp_aff)
    ref_lam_dm = emb_dm.lambdas_ 
    ref_PC_dm = emb_dm.maps_ 

    for i in file_list:
        print(f'{np.where(file_list==i)[0].item()}', '', end='', flush=True)
        dat = np.load(join(i,folder,f'aff_conn_mat_pear_top{Topk}_BNA_rmsubcor.npy'))

        emb_dm.fit(dat)
        lam, grad = [None]*1, [None]*1
        lam[0], grad[0] = emb_dm.lambdas_ , emb_dm.maps_

        pa = gradient.ProcrustesAlignment(n_iter=10)
        pa.fit(grad, reference=ref_PC_dm)
        aligned = np.array(pa.aligned_)

        np.save(join(i,folder,f'gradients_DM_top{Topk}_BNA_rmsubcor.npy'), aligned)

# Make sctx mean included FC eNkI

In [ ]:
demo = pd.read_excel(join(main_path, 'Enhanced_NKI_sorted.xlsx'), sheet_name='n=424', skiprows=1)
sub_list = demo['ID']
age = demo['Age']
sex = demo['Sex (M1F2)']

file_list = []


file_list = [join(main_path,'data',i) for i in sub_list]

len(file_list)

In [ ]:
# make mean sctx FC file

for i in file_list:
    print(f'{file_list.index(i)+1} / {len(file_list)}')
    
    ts = np.load(join(i,'ts_BNA.npy'))
    ts_ctx = ts[:210]
    
    # sctx list
    ts_amyg = ts[210:214].mean(axis=0)
    ts_hippo = ts[214:218].mean(axis=0)
    ts_cau = ts[[218,219,226,227]].mean(axis=0)
    ts_palli = ts[220:222].mean(axis=0)
    ts_accum = ts[222:224].mean(axis=0)
    ts_puta = ts[[224,225,228,229]].mean(axis=0)
    ts_thal = ts[230:].mean(axis=0)
    
    ts_mean_sctx = np.vstack((ts_amyg, ts_hippo, ts_cau, ts_palli, ts_accum, ts_puta, ts_thal))
    
    ts_whole_Msctx = np.concatenate((ts_ctx, ts_mean_sctx), axis=0)
    
    np.save(join(i, 'ts_whole_Msctx_BNA.npy'), ts_whole_Msctx)
    print('save ts')
    
    n_roi=217 # ctx 210 + sctx 7
    conn_mat =np.nan_to_num(np.where(np.eye(n_roi) ==1, 0, np.corrcoef(ts_whole_Msctx)))
    conn_mat_rtoz =  np.nan_to_num(np.arctanh(conn_mat),nan=0.0)
    
    np.save(join(i,'conn_mat_pear_whole_Msctx_BNA.npy'), conn_mat)
    print('save conn mat')
    np.save(join(i,'conn_mat_pear_rtoz_whole_Msctx_BNA.npy'), conn_mat_rtoz)
    print('save conn mat r to z')
    print('')

In [ ]:
# make sctx to ctx connection DC

sparsity=0.9
k = str(int(100-sparsity*100))
k = k.zfill(2)
print(f'Top {k}')
folder_name = f'top{k}'

for i in file_list:
    print(f'{file_list.index(i)+1} / {len(file_list)}')
    conn_mat_rtoz = np.load(join(i, 'conn_mat_pear_rtoz_whole_Msctx_BNA.npy'))[:,:210]
#     print(conn_mat_rtoz.shape)
    
    noaff_conn_mat = gradient.compute_affinity(np.nan_to_num(conn_mat_rtoz, nan=0.0), sparsity=sparsity)
    DC = noaff_conn_mat.sum(axis=1) # row-wise sum
    np.save(join(i, folder_name, f'noaff_conn_mat_pear_top{k}_Msctx2ctx_BNA.npy'),noaff_conn_mat)
    np.save(join(i, folder_name, f'DC_BNA_Msctx2ctx.npy'), DC)

In [ ]:
DC_file = np.array([np.load(join(i, 'top10', f'DC_BNA_Msctx2ctx.npy')) for i in file_list])
print(DC_file.shape)

In [ ]:
# DC age, sex regressed out

import statsmodels as sm
from statsmodels.api import GLM

list_reg_out = []

for i in range(DC_file.shape[1]):
    print(i, ' ', end='', flush=True)
    a = GLM(DC_file[:,i],sm.api.add_constant(np.array([sex,age,sex*age]).T))
    
    res_a = a.fit()
    # res.summary()

    list_reg_out.append(res_a.resid_response)
     
DC_regout = np.array(list_reg_out).T

In [ ]:
for i,x in enumerate(file_list):
    print(f'{i+1} / {len(file_list)}')
    np.save(join(x, 'top10', f'DC_BNA_Msctx2ctx_regout.npy'), DC_regout[i])

# Make gradient eNKI

In [ ]:
demo = pd.read_excel(join(main_path, 'Enhanced_NKI_sorted.xlsx'), sheet_name='n=424', skiprows=1)
sub_list = demo['ID']

file_list = []


file_list = [join(main_path,'data',i) for i in sub_list]

len(file_list)

In [ ]:
# FC thrsholding

for sparsity in [0.5, 0.8, 0.7, 0.6]:

#     sparsity = 0.75 # 0.75 0.5 0.8 0.7 0.6

    k = str(int(100-sparsity*100))
    k = k.zfill(2)

    print(f'Top {k}')

    folder_name = f'top{k}'

    for i in file_list:
        print(f'{file_list.index(i)} / {len(file_list)}')

        if not (os.path.isdir(os.path.join(i,f'{folder_name}'))):
            os.makedirs(os.path.join(i,f'{folder_name}'))

        conn_mat = np.load(join(i,'conn_mat_pear_rtoz_BNA.npy'))[:210, :210]

        noaff_conn_mat = gradient.compute_affinity(conn_mat, sparsity = sparsity)
        aff_conn_mat = gradient.compute_affinity(conn_mat, kernel = 'cosine', sparsity = sparsity)

        print(f'noaff isnan : {np.isnan(noaff_conn_mat).sum()}, symmetric : {gradient.is_symmetric(noaff_conn_mat)}')
        print(f'aff isnan : {np.isnan(aff_conn_mat).sum()}, symmetric : {gradient.is_symmetric(aff_conn_mat)}')

        np.save(join(i, folder_name, f'noaff_conn_mat_pear_top{k}_BNA_rmsubcor.npy'), noaff_conn_mat)
        np.save(join(i, folder_name, f'aff_conn_mat_pear_top{k}_BNA_rmsubcor.npy'), aff_conn_mat)

In [ ]:
# Make eigenvector

for i in [50, 20, 30, 40]:

    Topk = str(i) # 25 50 20 30 40
    folder = f'top{Topk}'

    grp_noaff = np.load(join(save_path, f'noaff_grp_conn_mat_pear_top{Topk}_BNA_rmsubcor.npy'))
    grp_aff = np.load(join(save_path, f'aff_grp_conn_mat_pear_top{Topk}_BNA_rmsubcor.npy'))

    comp_num = 10

    # PCA
    print('PCA')
    emb_pca = gradient.embedding.PCAMaps(n_components = comp_num)
    emb_pca.fit(grp_noaff)
    ref_lam_pca = emb_pca.lambdas_ 
    ref_PC_pca = emb_pca.maps_ 

    for i in file_list:
        print(f'{file_list.index(i)}', '', end='', flush=True)
        dat = np.load(join(i,folder,f'noaff_conn_mat_pear_top{Topk}_BNA_rmsubcor.npy'))

        emb_pca.fit(dat)
        lam, grad = [None]*1, [None]*1
        lam[0], grad[0] = emb_pca.lambdas_ , emb_pca.maps_

        pa = gradient.ProcrustesAlignment(n_iter=10)
        pa.fit(grad, reference=ref_PC_pca)
        aligned = np.array(pa.aligned_)

        np.save(join(i,folder,f'gradients_PCA_top{Topk}_BNA_rmsubcor.npy'), aligned)

    print('\n')

    # Diffusion map
    print('Diffusion Map')
    emb_dm = gradient.embedding.DiffusionMaps(n_components = comp_num)
    emb_dm.fit(grp_aff)
    ref_lam_dm = emb_dm.lambdas_ 
    ref_PC_dm = emb_dm.maps_ 

    for i in file_list:
        print(f'{file_list.index(i)}', '', end='', flush=True)
        dat = np.load(join(i,folder,f'aff_conn_mat_pear_top{Topk}_BNA_rmsubcor.npy'))

        emb_dm.fit(dat)
        lam, grad = [None]*1, [None]*1
        lam[0], grad[0] = emb_dm.lambdas_ , emb_dm.maps_

        pa = gradient.ProcrustesAlignment(n_iter=10)
        pa.fit(grad, reference=ref_PC_dm)
        aligned = np.array(pa.aligned_)

        np.save(join(i,folder,f'gradients_DM_top{Topk}_BNA_rmsubcor.npy'), aligned)


In [ ]:
grp_aff = np.load(join(path_subj, f'aff_grp_conn_mat_pear_top10_BNA_rmsubcor.npy'))

comp_num = 10

emb_dm = gradient.embedding.DiffusionMaps(n_components = comp_num)
emb_dm.fit(grp_aff)
ref_lam_dm = emb_dm.lambdas_ 
ref_PC_dm = emb_dm.maps_ 

sns.barplot(x = list(range(1,11) ), y = ref_lam_dm/ref_lam_dm.sum(), palette='Greys_r') # 'ch:2,r=1,l=.6' Greys_r
plt.xlabel('Principal component', fontsize = 15)
plt.ylabel('The ratio of variance', fontsize = 15)
plt.title('Explained Variance')
plt.grid(False)

print(ref_lam_dm/ref_lam_dm.sum())

# Clustering

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler, OneHotEncoder

age = demo['Age']
sex = demo['Sex (M1F2)']
BMI = demo['BMI']
WHR = demo['WHR']
EDE_Q_R = demo['Restraint']
EDE_Q_E_con = demo['Eating concern']
EDE_Q_S_con = demo['Shape concern']
EDE_Q_W_con = demo['Weight concern']
TFEQ_F1 = demo['Factor 1']
TFEQ_F2 = demo['Factor 2']
TFEQ_F3 = demo['Factor 3']

EDE_Q_global = (EDE_Q_R+EDE_Q_E_con+EDE_Q_S_con+EDE_Q_W_con) / 4
EDE_Q_con = (EDE_Q_E_con+EDE_Q_S_con+EDE_Q_W_con) / 3

scaler = MinMaxScaler() # MinMaxScaler StandardScaler MaxAbsScaler RobustScaler
 
y_target = np.array(BMI)
y_scaled = scaler.fit_transform(np.array(y_target).reshape(-1,1)).reshape(-1)

demographic_label = ['age', 'BMI', 'WHR', 'EDE_Q_R', 'EDE_Q_E_con', 'EDE_Q_S_con', 'EDE_Q_W_con', 'TFEQ_F1', 'TFEQ_F2', 'TFEQ_F3']

for i, x in enumerate([age, BMI, WHR, EDE_Q_R, EDE_Q_E_con, EDE_Q_S_con, EDE_Q_W_con, TFEQ_F1, TFEQ_F2, TFEQ_F3]):
    print(demographic_label[i], ':', np.array(x).shape, np.round(x.mean(),2), np.round(x.std(),2), '      Min Max', np.round(x.min(),2), np.round(x.max(),2))
    
print('Sex F M : ',len(np.where(sex==2)[0]), len(np.where(sex==1)[0]))

In [ ]:
print(len(np.where((BMI>=18.5)&(BMI<25))[0]))
print(len(np.where((BMI>=25)&(BMI<30))[0]))
print(len(np.where(BMI>=30)[0]))

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler, OneHotEncoder
# Lemon

age_lemon = demo_lemon['Age']
sex_lemon = demo_lemon['Gender_ 1=male_2=female']
BMI_lemon = demo_lemon['BMI (kg/m2)']
TFEQ_F1_lemon = demo_lemon['FEV_KK']
TFEQ_F2_lemon = demo_lemon['FEV_STOER']
TFEQ_F3_lemon = demo_lemon['FEV_HUNGER']

scaler = MinMaxScaler() # MinMaxScaler StandardScaler MaxAbsScaler RobustScaler
 
y_target = np.array(BMI_lemon)
y_scaled = scaler.fit_transform(np.array(y_target).reshape(-1,1)).reshape(-1)
    
demographic_label = ['age', 'BMI', 'TFEQ_F1', 'TFEQ_F2', 'TFEQ_F3']

for i, x in enumerate([age_lemon, BMI_lemon, TFEQ_F1_lemon, TFEQ_F2_lemon, TFEQ_F3_lemon]):
    print(demographic_label[i], ':', np.array(x).shape, np.round(x.mean(),2), np.round(x.std(),2), '      Min Max', np.round(x.min(),2), np.round(x.max(),2))
    
print('Sex F M : ',len(np.where(sex_lemon==2)[0]), len(np.where(sex_lemon==1)[0]))

In [ ]:
print(len(np.where((BMI_lemon>=18.5)&(BMI_lemon<25))[0]))
print(len(np.where((BMI_lemon>=25)&(BMI_lemon<30))[0]))
print(len(np.where(BMI_lemon>=30)[0]))

# Demographinc statistic

In [ ]:
Dis_demo = [age, BMI, TFEQ_F1, TFEQ_F2, TFEQ_F3]
Rep_demo = [age_lemon, BMI_lemon, TFEQ_F1_lemon, TFEQ_F2_lemon, TFEQ_F3_lemon]
demographic_label = ['age', 'BMI', 'TFEQ_F1', 'TFEQ_F2', 'TFEQ_F3']

for i in range(len(Dis_demo)):

    [s_0,p_0] = sc.stats.ttest_ind(Dis_demo[i], Rep_demo[i], equal_var = False, axis=0)
    print(f'{demographic_label[i]} : {p_0}')

In [ ]:
# discovery, replication dataset sex chi-square

chi_ob = [282, 142] # discovery(eNKI) F:M
chi_expect = [75,137] # replication(Lemon) F:M

chi_stat, chi_pval = sc.stats.chisquare(chi_ob, f_exp=chi_expect, axis=None, ddof = [0,1,2])

print(chi_stat)
print(chi_pval)

In [ ]:
path_data = join(main_path,'data')
path_data_Lemon = join(main_path,'data_Lemon')

topk = 'top10' # top10 top20 top25 top30 top40 top50

file_list = np.concatenate([np.load(join(path_data,i, topk, f'gradients_DM_{topk}_BNA_rmsubcor.npy')) for i in sub_list], axis=0) # gradients_PCA_top10_BNA gradients_DM_top10_BNA
# file_list = np.concatenate([np.load(join(path_data_Lemon,i, topk, f'gradients_DM_{topk}_BNA_rmsubcor.npy')) for i in sub_list_lemon], axis=0) # gradients_PCA_top10_BNA gradients_DM_top10_BNA


grad1_list = file_list[:,:,0]
grad2_list = file_list[:,:,1]
grad3_list = file_list[:,:,2]

# gradient z normalized

grad1_list_znorm = sc.stats.zscore(grad1_list, axis=1)
grad2_list_znorm = sc.stats.zscore(grad2_list, axis=1)
grad3_list_znorm = sc.stats.zscore(grad3_list, axis=1)

# gradient concatenate
grad_list_znorm_stacked = np.concatenate((grad1_list_znorm, grad2_list_znorm, grad3_list_znorm), axis=1)

In [ ]:
# gradient age, sex regressed out

import statsmodels as sm
from statsmodels.api import GLM

list_reg_out_a = []
list_reg_out_b = []
list_reg_out_c = []
list_reg_out_d = []

for i in range(grad_list_znorm_stacked.shape[1]):
    print(i, ' ', end='', flush=True)
    a = GLM(grad_list_znorm_stacked[:,i],sm.api.add_constant(np.array([sex,age,sex*age]).T))
    
    res_a = a.fit()
    # res.summary()

    list_reg_out_a.append(res_a.resid_response)
     
grad_list_znorm_stacked_regout = np.array(list_reg_out_a).T


for i in range(grad1_list_znorm.shape[1]):
    print(i, ' ', end='', flush=True)
    b = GLM(grad1_list_znorm[:,i],sm.api.add_constant(np.array([sex,age,sex*age]).T))
    c = GLM(grad2_list_znorm[:,i],sm.api.add_constant(np.array([sex,age,sex*age]).T))
    d = GLM(grad3_list_znorm[:,i],sm.api.add_constant(np.array([sex,age,sex*age]).T))

    res_b = b.fit()
    res_c = c.fit()
    res_d = d.fit()
    # res.summary()

    list_reg_out_b.append(res_b.resid_response)
    list_reg_out_c.append(res_c.resid_response)
    list_reg_out_d.append(res_d.resid_response)
    
    
grad1_list_znorm_regout = np.array(list_reg_out_b).T
grad2_list_znorm_regout = np.array(list_reg_out_c).T
grad3_list_znorm_regout = np.array(list_reg_out_d).T

In [ ]:
grad1_DM_temp = np.load(join(main_path, 'data_HCP', 'grad1_DM_top10_temp_BNA_rmsubcor.npy'))[:210]
grad2_DM_temp = np.load(join(main_path, 'data_HCP', 'grad2_DM_top10_temp_BNA_rmsubcor.npy'))[:210]
grad3_DM_temp = np.load(join(main_path, 'data_HCP', 'grad3_DM_top10_temp_BNA_rmsubcor.npy'))[:210]
grad_DM_temp = np.array([grad1_DM_temp, grad2_DM_temp, grad3_DM_temp]).T

grad_list_znorm_stacked_regout_3d = np.dstack((grad1_list_znorm_regout, grad2_list_znorm_regout, grad3_list_znorm_regout))

# Manifold Eccentricity
ncompo = 3

nroi = grad_DM_temp.shape[0]
gm_center = grad_DM_temp[:,:ncompo].mean()
ME = np.array([np.sqrt(np.sum(np.square(gm_center-file_list[i,:,:ncompo]), axis=1)) for i in range(len(file_list))])
ME_regout = np.array([np.sqrt(np.sum(np.square(gm_center-grad_list_znorm_stacked_regout_3d[i,:,:ncompo]), axis=1)) for i in range(len(grad_list_znorm_stacked_regout_3d))])